# HW2. NGS data processing and mutation calling

In this homework we will be taking raw sequencing data in fastq format (as one might recieve from a sequencing center) and performing alignment and mutation calling. For the dataset we are using data simulated as part of the ICGC-TCGA DREAM challenge, in which we know certain somatic events were spiked in to the tumor sample (and not the normal). The caller will need to identify these events, and distinguish them from germline mutations and other artifactual signals. To lessen the computational load, we've restricted to reads mapping to chromosome 6 (in the original alignment) and will only call mutations there. Files were sourced and modified from below

https://github.com/bcbio/bcbio-nextgen/tree/master/config/teaching

To complete this homework, you will need to install the following tools:

- IGV https://software.broadinstitute.org/software/igv/
- samtools http://www.htslib.org/download/
- bwa https://github.com/lh3/bwa

Included in the data/ folder are the following files:

- cancer-syn3-chr6-[tumor/normal]-[1/2].fq.gz: gzipped fastq files for Read 1 and Read 2(paired end sequencing) of a simulated tumor and normal exome pair. 
- NGv3-chr6-hg19.noHLA.bed: A list of exonic regions on which you should perform your mutation calling. Restricted to choromosome 6, and the region with HLA genes is removed (it is notoriously tricky due to the high levels of genetic variation in those genes).
- ground_truth_mutations.txt: list of somatic mutations that were spiked in as part of the simulation.
- Homo_sapiens_assembly19.fasta*: The human reference genome (build hg19) and bwa's pre-built index files

## (Question 1) Alignment

### (1a) By examining the fastq files with unix tools, determine the read lengths of R1 and R2 of this sequencing data. (Remember they are just text files that have been compressed with gzip)

In [2]:
%%bash

# You can use the %%bash cell magic to run blocks of bash commands within the notebook

### (1b) Align the data to human genome build hg19 using bwa-mem and the reference files and pre-built indices included. Use "samtools sort" to obtain a coordinate-sorted bam file, and "samtools index" to index the bam.

### (1c) Print the first 10 reads in the bam file using "samtools view"

### (1d) Use samtools idxstats to count the number of reads on each chromosome to confirm reads mostly align to chromosome 6

## (2) Manual exploration

#### Using IGV (https://software.broadinstitute.org/software/igv/) and your newly aligned bams, examine the following sites. For each site, show a screenshot and argue if you think there is a somatic, germline, or no mutation. 

- Site 1 chr6:32261771
- Site 2 chr6:31922356
- Site 3 chr6:107113731

## (3) Mutation calling

### (3a) Write a simple mutation caller that iterates over each position, and calculates the $LOD_T$ the log likelihood ratio of a somatic mutation vs noise. For each candidation passing $\theta_T = 6.3$. For each variant, calculate whether $LOD_N$ passes $\theta_N=2.2$. Report any mutations passing as somatic mutation calls.

### To get you started, we have written a loop that iterates over the bam file using pysam and identifies the bases and qualities at every pileup. Mutations only should be considered within the exome calling intervals. If you are using a language other than python, you can either choose to adapt this code using a package in your language, or use samtools to genate a text file with the bases and qualities at each positions.

samtools mpileup -A -q30 -f Homo_sapiens_assembly19.fasta -l NGv3-chr6-hg19.bed [your_bam_file] > [your_mpileup_file.txt]

### Then process this text file in the language of your choice. Make sure to review the samtools docs for interpretation of the output.

In [4]:
import pysam
import pandas as pd
import numpy as np
from collections import Counter

In [16]:
# Given a pileupcolumn object, returns list of bases 
def get_bases_and_qualities(pileupcolumn):

    bases = pileupcolumn.get_query_sequences()
    quals = pileupcolumn.get_query_qualities()
    
    bases = np.char.upper(np.array(bases))
    quals = np.array(quals)
    
    return bases,quals

In [17]:
# For mutation ref_base -> alt occuring at allele fraction f,
# calculate the LOD given bases and qualities at a pileup
def calc_LOD(bases,quals,ref_base,alt,f):
    ### To do: Implement this function
    pass
        


In [20]:
# Load intervals to call
I = pd.read_csv('data/NGv3-chr6-hg19.noHLA.bed',header=None,sep='\t',names=['chrom','st','en'])
I['chrom'] = I['chrom'].astype(str)
I.head()

,chrom,st,en
0,6,105670,106956
1,6,123624,124168
2,6,131690,132266
3,6,146097,146761
4,6,203140,204023


In [104]:
## Step 1: find candidate variants in tumor

# Open files
samfile = pysam.AlignmentFile("data/tumor.bam", "rb")
normalfile = pysam.AlignmentFile("data/normal.bam", "rb")
reffile = pysam.FastaFile("data/Homo_sapiens_assembly19.fasta")

# LOD_T threshhold 
theta_T = 6.3

# List to append variants as we find them
variants = list()


chrom="6"
st = I['st'].min()
en = I['en'].max()

# Load reference genome over required interval
ref = reffile.fetch(chrom,st,en)

# Index to keep track of exon
i=0
curr_st = I['st'].iloc[0]
curr_en = I['en'].iloc[0]


# Iterate over pileups in this region
# Hint: for debugging purposes you may want to start just running in the vicinity of a known true event
for pileupcolumn in samfile.pileup(chrom, st, en,min_mapping_quality=30,ignore_orphans=False):
    
        # Check if we're in our exon intervals, if not continue
        if pileupcolumn.reference_pos <curr_st:
            continue
        elif pileupcolumn.reference_pos>=curr_en:
            # and if we've passed the current exon, we need to increment our index
            while pileupcolumn.reference_pos>=curr_en:
                i+=1
                curr_st = I['st'].iloc[i]
                curr_en = I['en'].iloc[i]

        # Get the reference base at the pileup position
        ref_base = ref[pileupcolumn.reference_pos-st]    

    
        # Get the bases of reads covering the base, and their base qualities
        bases,quals = get_bases_and_qualities(pileupcolumn)
    
        
        # If we don't have at least 2 reads, skip
        if len(quals)<2:
            continue
    
        ### You do the rest! Figure out which mutation mutation you want to test for, 
        ###   calculate LOD_T, and decide if it passes theta_T
            
 
                
samfile.close()
normalfile.close()
reffile.close()

In [106]:
## STEP 2: Annotate normal LOD

normalfile = pysam.AlignmentFile("data/normal.bam", "rb")

# Iterate over calls (assuming you've made a dataframe of variants V)
for ind,row in V.iterrows():
    # "Loops" over single pileup at our candidate variants
    for n in normalfile.pileup("6",row['pos'],row['pos']+1,
                               min_mapping_quality=30,truncate=True,ignore_orphans=False):
        
        # Get the normal bases and qualities
        bases_n,quals_n = get_bases_and_qualities(n)
        
        ## TODO: Calculate the normal LOD
        ## Decide if it's a real variant
        
normalfile.close()

    

### (3b) Show how many variants you discovered. Using the ground truth mutations provided, calculate your callers sensitivity and false positive rate

In [5]:
G = pd.read_csv('data/ground_truth_mutations.txt',sep='\t',header=None,
               names = ['chr','pos','ref','alt'])
G.head()

,chr,pos,ref,alt
0,6,4999924,G,C
1,6,17781103,T,A
2,6,18171698,T,A
3,6,42832434,T,C
4,6,51946892,G,C


### (3c) Explore in IGV examples of false positive calls by your caller, as well as false negatives. In each case include screenshots, and describe your findings.